### Import Dependencies

In [1]:
import openai
import instructor
from pydantic import BaseModel, Field
from qdrant_client import QdrantClient

#### Mock Example

In [2]:
prompt = """
You are a helpful assistant.
Return an answer to the question.
The question is: What's your name?
"""

In [3]:
response = openai.chat.completions.create(
    model="gpt-4.1-mini", messages=[{"role": "system", "content": prompt}]
)

In [4]:
print(response.choices[0].message.content)

I’m ChatGPT. How can I assist you today?


#### Add Instructor - Structured Outputs

In [3]:
client = instructor.from_openai(openai.OpenAI())

In [4]:
class RAGGenerationResponse(BaseModel):
    answer: str = Field("Answer to the question")

In [7]:
response, raw_response = client.chat.completions.create_with_completion(
    model="gpt-4.1-mini",
    messages=[{"role": "system", "content": prompt}],
    response_model=RAGGenerationResponse,
)

In [8]:
response

RAGGenerationResponse(answer='I am ChatGPT, your AI assistant. How can I help you today?')

In [9]:
response.model_dump()

{'answer': 'I am ChatGPT, your AI assistant. How can I help you today?'}

### RAG Example

In [2]:
import sys

sys.path.insert(0, "../..")

from src.api.rag.retrieval_generation import (
    rag_pipeline,
)

In [3]:
qdrant_client_ = QdrantClient(url="http://localhost:6333")

output = rag_pipeline("Can i get a tablet?", qdrant_client_)

In [4]:
output

{'answer': RAGGenerationResponse(answer='There are no tablets available in the current selection of products. However, there are accessories compatible with tablets such as iPad cases and stylus pens, but no standalone tablets are listed.'),
 'question': 'Can i get a tablet?',
 'retrieved_context_ids': ['B0BF18F6R7',
  'B08BX2L8F2',
  'B09QGNB537',
  'B0C9XFF3CT',
  'B0BG5L2YLC'],
 'retrieved_context': ['KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges w